# Immune microenvironment analysis (Supp Fig 8 and Fig 5)

## T cell resuce from doublets

In [ ]:
library(Seurat)
library(SeuratObject)
library(ggplot2)
library(circlize)
library(tidyverse)
library(patchwork)
library(qs)
library(reticulate)
library(harmony)
library(RColorBrewer)
library(MAST)
library(bluster)
library(BPCells)
library(ggrepel)
library(ComplexHeatmap) 

xenium_anno <- read.csv("path/to/initial_cell_typing.csv" )
xenium_all <- qread("path/to/xenium_merged_obj.qs")
xenium_all <- NormalizeData(xenium_all)
p1 <- FeatureScatter(xenium_all, feature1="CD3E", feature2="CD8A") + NoLegend()
CD8_Tcells <- row.names(p1$data %>% filter(CD3E > 2 & CD8A > 2))
 
p2 <- FeatureScatter(xenium_all, feature1="CD3E", feature2="CD4") + NoLegend()
CD4_Tcells <- row.names(p2$data %>% filter(CD3E > 2 & CD4 > 2))
length(CD4_Tcells)

xenium_all[[]] <- xenium_all[[]] %>% mutate(T_celltype = case_when(All_cell_type2 == "NK" ~ "other",
                                                                                 All_cell_type2 %in% c("CD4_Resting_T", "CD4_Th1-like", "CD4_Th17_T", "CD4_Treg" , "CD4_naive/CM_T") ~ "CD4_Tcells", 
                                                                                 All_cell_type2 %in% c("CD8_TRM_T", "CD8_Effector_T") ~ "CD8_Tcells", 
                                                                                 (barcode %in% CD8_Tcells & (All_cell_type2 %in% c("Doublet", "Mesen_T_doublet", "CD4_CD8_doublet"))) ~ "CD8_Tcells",
                                                                                 (barcode %in% CD4_Tcells & (All_cell_type2 %in% c("Doublet", "Mesen_T_doublet", "CD4_CD8_doublet"))) ~ "CD4_Tcells",
                                                                                 All_cell_type2 == "Proliferative_T" ~ "Proliferative_T",
                                                                                 TRUE ~ "other"))
Idents(xenium_all) = "T_celltype"
xenium_CD4T <- subset(xenium_all, T_celltype == "CD4_Tcells")

Tregs_norm <- FetchData(object = xenium_CD4T, vars =  c('IL2RA','FOXP3','TIGIT','CTLA4', 'CCR4'), layer = "data")
filtered <- Tregs_norm %>%  filter(FOXP3 > 2) 
filtered

xenium_all@meta.data <- xenium_all@meta.data  %>% mutate(CD4Treg = if_else(barcode %in% rownames((filtered) | All_cell_type2 == "CD4_Treg"), "CD4_Tregs", "Not"))
   
xenium_all[[]] <- xenium_all[[]] %>% mutate(All_cell_type3 = case_when(T_celltype == "CD8_Tcells" ~ "CD8_Tcells",
                                                                     CD4Treg == "CD4_Tregs" ~ "CD4_Tregs",
                                                                     T_celltype == "CD4_Tcells" ~ "other_CD4_Tcells",
                                                                     TRUE ~ All_cell_type2))

                                                                  




## Supp Fig 8a

In [ ]:
xenium_anno <- xenium_anno %>% mutate(site = case_when((tn_label == 0 & tns_label != 0 & !is.na(tns_label)) ~ "PSI",
                                                       (tn_label != 0 & !is.na(tn_label)) ~ "TMR",
                                                        (tn_label == 0 & tns_label == 0) ~ "APT",
                                                             TRUE ~ "other"))     

xenium_anno %>% group_by(site) %>% summarise(n=n())
xenium_anno$site2 <- factor(xenium_anno$site, levels = c("TMR", "PSI", "APT"))

xenium_anno <- xenium_anno %>% mutate(new_cell_type = case_when(All_cell_type3 %in% c("CD8_Tcells") ~ "CD8_Tcell",
                                                All_cell_type3 %in% c( "other_CD4_Tcells", "CD4_Tregs") ~ "CD4_Tcell",               
                                                 All_cell_type3 %in% c("cDC1", "cDC2", "pDC", "LAMP3_DC") ~ "DCs",
                                                All_cell_type3 %in% c("TAM", "M2_TAM", "IFN_TAM" ,"SPP1_TAM" ) ~ "TAM",
                                                All_cell_type3 %in% c("classical_monocyte", "PMN-like_monocyte") ~ "monocyte",
                                                TRUE ~ All_cell_type3))

Immune = c("B_cell", "CD4_CD8_doublet",  "CD8_Tcells", "CD4_Tregs", "IFN_TAM", "LAMP3_DC", "M2_TAM", "NK", "PMN-like_monocyte",  "Plasma_cell", "Proliferative_T", "SPP1_TAM", "TAM", "cDC1", "cDC2", "classical_monocyte", "other_CD4_Tcells", "pDC" )

Plot <- xenium_anno %>% filter(site == "TMR" & Organ %in% c( "Colon","Liver", "Lung") & All_cell_type3 %in% Immune & sample_id != "HT413C1-Th1K2A1U2" & sample_id != "CM626C1-A7-S1Fp1U1") %>%  group_by(Organ, Case_ID, sample_id, new_cell_type) %>% summarise(n=n())  %>% mutate(prop = n/sum(n)) %>% filter(new_cell_type %in% c("CD8_Tcell", "TAM", "CD4_Tcell", "B_cell", "NK"))
p1 <- Plot  %>% ggplot(aes(x=Organ, y=prop, fill = Organ, color = Organ)) + geom_boxplot(linewidth = 1.5, alpha = 0.5) + geom_point(aes(color = Organ), size = 3)  + scale_fill_manual(values=c('#C2B280', 'brown' , 'steelblue1')) + scale_color_manual(values=c('#C2B280', 'brown' , 'steelblue1')) + geom_line(aes(group = Case_ID), color = "black")  +   theme_classic() +  theme(axis.text=element_text(size=15))+  facet_wrap(.~new_cell_type,  scales = "free") 

Plot <- xenium_anno %>% filter(site == "PSI" & Organ %in% c( "Colon","Liver", "Lung") & All_cell_type3 %in% Immune & sample_id != "HT413C1-Th1K2A1U2" & sample_id != "CM626C1-A7-S1Fp1U1") %>%  group_by(Organ, Case_ID, sample_id, new_cell_type) %>% summarise(n=n())  %>% mutate(prop = n/sum(n)) %>% filter(new_cell_type %in% c("CD8_Tcell", "TAM", "CD4_Tcell", "B_cell", "NK"))
p2 <- Plot  %>% ggplot(aes(x=Organ, y=prop, fill = Organ, color = Organ)) + geom_boxplot(linewidth = 1.5, alpha = 0.5) + geom_point(aes(color = Organ), size = 3)  + scale_fill_manual(values=c('#C2B280', 'brown' , 'steelblue1')) + scale_color_manual(values=c('#C2B280', 'brown' , 'steelblue1')) + geom_line(aes(group = Case_ID), color = "black")  +   theme_classic() +  theme(axis.text=element_text(size=15))+  facet_wrap(.~new_cell_type,  scales = "free") 

Plot <- xenium_anno %>% filter(site == "APT" & Organ %in% c( "Colon","Liver", "Lung") & All_cell_type3 %in% Immune & sample_id != "HT413C1-Th1K2A1U2" & sample_id != "CM626C1-A7-S1Fp1U1") %>%  group_by(Organ, Case_ID, sample_id, new_cell_type) %>% summarise(n=n())  %>% mutate(prop = n/sum(n)) %>% filter(new_cell_type %in% c("CD8_Tcell", "TAM", "CD4_Tcell", "B_cell", "NK"))
p3 <- Plot  %>% ggplot(aes(x=Organ, y=prop, fill = Organ, color = Organ)) + geom_boxplot(linewidth = 1.5, alpha = 0.5) + geom_point(aes(color = Organ), size = 3)  + scale_fill_manual(values=c('#C2B280', 'brown' , 'steelblue1')) + scale_color_manual(values=c('#C2B280', 'brown' , 'steelblue1')) + geom_line(aes(group = Case_ID), color = "black")  +   theme_classic() +  theme(axis.text=element_text(size=15))+  facet_wrap(.~new_cell_type,  scales = "free") 


## Supp Fig 8b

In [ ]:
#mask_size_summary.csv is a file with area of tn_mask (tumor microregion mask area) and tns_mask (gross tumor region mask area) in 100microm2, and Total_area (fov area) manually obtained from xenium explorer in mm2
tn_area <- read.csv("/path/to/mask_size_summary.csv")
row.names(tn_area) <- tn_area$section_id
tn_area 

In [ ]:
#sample id HT413C1 is MSI high and CM626C1 is very small and can bias results
#This is the code for CD8 T cell density, switch CD8 T cell with TAM for TAM density

Plot1 <- xenium_anno %>% filter(site == "TMR" & Organ %in% c( "Colon", "Liver", "Lung") & new_cell_type %in% "CD8_Tcell"  & sample_id != "HT413C1" & sample_id != "CM626C1") %>% group_by(Organ, Case_ID, sample_id) %>% summarise(n=n())  
Plot2 <- xenium_anno %>% filter(site == "PSI" & Organ %in% c( "Colon", "Liver", "Lung") & new_cell_type %in% "CD8_Tcell" & sample_id != "HT413C1" & sample_id != "CM626C1") %>% group_by(Organ, Case_ID, sample_id) %>% summarise(n=n())  
Plot3 <- xenium_anno %>% filter(site == "APT" & Organ %in% c( "Colon", "Liver", "Lung") & new_cell_type %in% "CD8_Tcell" & sample_id != "HT413C1" & sample_id != "CM626C1") %>% group_by(Organ, Case_ID, sample_id) %>% summarise(n=n())  


In [ ]:
Plot1 <- as.data.frame(Plot1)
row.names(Plot1) <- Plot1$sample_id
merged_TMR_T <- merge(Plot1, tn_area, by=0)
merged_TMR_T <- merged_TMR_T  %>% mutate(prop_area = n*10000/tn_mask_size)
merged_TMR_T 
p1 <- merged_TMR_T  %>% ggplot(aes(x=Organ, y=prop_area, fill = Organ, color = Organ))  + geom_boxplot(linewidth = 1.5, alpha = 0.5) + geom_point(aes(color = Organ), size = 3)  + scale_fill_manual(values=c('#C2B280', 'brown' , 'steelblue1')) + scale_color_manual(values=c('#C2B280', 'brown' , 'steelblue1')) + geom_line(aes(group = Case_ID), color = "black") + scale_y_continuous(trans = "log2")  +   theme_classic() +  theme(axis.text=element_text(size=15))

Plot2 <- as.data.frame(Plot2)
row.names(Plot2) <- Plot2$sample_id
merged_PSI_T <- merge(Plot2, tn_area, by=0)
merged_PSI_T <- merged_PSI_T  %>% mutate(prop_area = n*10000/(tns_mask_size-tn_mask_size))
merged_PSI_T 
p2 <- merged_PSI_T  %>% ggplot(aes(x=Organ, y=prop_area, fill = Organ, color = Organ))  + geom_boxplot(linewidth = 1.5, alpha = 0.5) + geom_point(aes(color = Organ), size = 3)  + scale_fill_manual(values=c('#C2B280', 'brown' , 'steelblue1')) + scale_color_manual(values=c('#C2B280', 'brown' , 'steelblue1')) + geom_line(aes(group = Case_ID), color = "black") + scale_y_continuous(trans = "log2")  +   theme_classic() +  theme(axis.text=element_text(size=15))

#For APT analysis removing CM663C1 liver metastasis does not have much APT
Plot3 <- as.data.frame(Plot3)
row.names(Plot3) <- Plot3$sample_id
merged_APT_T <- merge(Plot3, tn_area, by=0)
rownames(merged_APT_T) <- merged_APT_T$sample_id
merged_APT_T  <- as.data.frame(merged_APT_T[,-1])
merged_APT_T  <- merged_APT_T  %>% mutate(tns_area = (tns_mask_size/10000))
merged_APT_T 
merged_APT_T$fov_area <- as.numeric(merged_APT_T$fov_area)
merged_APT_T <- merged_APT_T  %>% mutate(prop_area = n/(fov_area-tns_area))
p3 <- merged_APT_T  %>% filter(sample_id != "CM663C1") %>% ggplot(aes(x=Organ, y=prop_area, fill = Organ, color = Organ))  + geom_boxplot(linewidth = 1.5, alpha = 0.5) + geom_point(aes(color = Organ), size = 3)  + scale_fill_manual(values=c('#C2B280', 'brown' , 'steelblue1')) + scale_color_manual(values=c('#C2B280', 'brown' , 'steelblue1')) + geom_line(aes(group = Case_ID), color = "black") + scale_y_continuous(trans = "log2")  +   theme_classic() +  theme(axis.text=element_text(size=15))
p3

# Supp Fig 8c and d

In [ ]:
#Defining exhasuted CD8 T cells
Idents(xenium_all) = "All_cell_type3"
xenium_CD8T <- subset(xenium_all, All_cell_type3 == "CD8_Tcells")

exhaustion <- FetchData(xenium_CD8T, vars = c("PDCD1", "TIGIT", "LAG3", "HAVCR2", "CTLA4"), layer = "counts") 
exhaustion <- exhaustion %>% mutate(sum =  rowSums(pick(where(is.numeric))))
exhausted_CD8 = rownames(exhaustion %>% filter(sum >= 1))
xenium_CD8T[[]] <- xenium_CD8T[[]]  %>% mutate(exhausted_by_marker = if_else(barcode %in% exhausted_CD8, "Yes", "No"))

p2 = DotPlot(xenium_CD8T, features= markers, group.by="exhausted_by_marker") +
theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5)) + 
      coord_fixed() +
      scale_color_gradientn(colors = rdylbu_colors, limits = c(-2, 2), breaks = color_breaks) +  
      scale_size_area(limits = c(0, 100), oob = scales::squish)


xenium_T <- xenium_CD8T[[]] %>% filter(site2 %in% c("TMR","PSI" , "APT") & Organ %in% c("Colon", "Liver", "Lung") &  sample_id != "HT413C1" & sample_id != "CM626C1" ) %>% group_by(Organ,  sample_id, site2, exhausted_by_marker) %>% summarise(n=n()) %>% mutate(prop = n/sum(n)) %>% filter(exhausted_by_marker == "Yes") 
p1 <- xenium_T %>%  ggplot(aes(x=site2, y=prop, fill = site2, color = site2)) + geom_boxplot(linewidth = 1.5, alpha = 0.5)  + scale_fill_manual(values=c('red','green3', 'blue')) + scale_color_manual(values=c('red','green3', 'blue')) +  geom_line(aes(group = sample_id), color = "black")  +  facet_wrap(.~Organ) + theme_classic() +  theme(axis.text=element_text(size=15))  


# Chemokine spatial expression

In [ ]:
#List of chemokine genes
chemokines = c(Features(xenium_all)[grepl(c("CXCL") , Features(xenium_all))] , Features(xenium_all)[grepl(c("CCL") , Features(xenium_all))], Features(xenium_all)[grepl(c("CX3CL") , Features(xenium_all))])
chemokines

#Subset xenium object by liver and lung non tumor cells
Idents(xenium_all) <- "Organ"
xenium_chemokine <- subset(xenium_all, ident = c("Lung", "Liver"), features = chemokines)        

Idents(xenium_chemokine) <- "Broad_cell_type2"
xenium_chemokine <- subset(xenium_chemokine, ident = "Tumor", invert = TRUE)

#Define TMR, PSI and APT
xenium_anno <- xenium_anno %>% mutate(site = case_when((tn_label == 0 & tns_label != 0 & !is.na(tns_label)) ~ "PSI",
                                                       (tn_label != 0 & !is.na(tn_label)) ~ "TMR",
                                                        (tn_label == 0 & tns_label == 0) ~ "APT",
                                                             TRUE ~ "other"))     

xenium_anno %>% group_by(site) %>% summarise(n=n())
xenium_anno$site2 <- factor(xenium_anno$site, levels = c("TMR", "PSI", "APT"))

#subset by APT or non-APT
Idents(xenium_chemokine) <- "site"
xenium_chemokine_nonAPT <- subset(xenium_chemokine, ident = c("TMR", "PSI"))
xenium_chemokine_APT <- subset(xenium_chemokine, ident = c("APT"))

xenium_chemokine_nonAPT <- NormalizeData(xenium_chemokine_nonAPT)
xenium_chemokine_APT <- NormalizeData(xenium_chemokine_APT)

#Obtain average expression of chemokine across all non tumor cells in APT and nonAPT for each sample
chemokine_nonAPT <- AverageExpression(xenium_chemokine_nonAPT, group.by = c("Organ", "sample_id", "Case_ID"), return.seurat = FALSE, layer = "data")
chemokine_APT <- AverageExpression(xenium_chemokine_APT, group.by = c("Organ", "sample_id", "Case_ID"), return.seurat = FALSE, layer = "data")

#Analysis and Figure script (showing here for chemokine_APT)
#CM663C not included in this analysis as that sample does not have APT
 
chemokine_APT <- as.data.frame(t(chemokine_APT))
chemokine_APT$sample <- rownames(chemokine_APT)
chemokine_APT <- chemokine_APT %>% separate(col = sample,  into = c("Organ", "sample_id", "Case_ID"), sep = "_" )
head(chemokine_APT)
Heatmap_chemokine <- chemokine_APT[,-c(31,32, 33)]         
Heatmap_chemokine <- t(scale(Heatmap_chemokine))
quantile(Heatmap_chemokine, c(0.1, 0.95))
col_fun = colorRamp2(c(-0.9, 0, 1.8), c("blue", "white", "red"))

wilcox <- lapply(1:30, function(x) wilcox.test(chemokine_metadata[[x]]~chemokine_metadata$Organ))
p_val <- (sapply(wilcox, function(x) x$p.value))
top_diff = which(p_val <= 0.0005) 

colnames(Heatmap_chemokine) <- chemokine_metadata$Case_ID
chemokine_metadata <- chemokine_APT[,c(31,33)]

# Supp Fig 8e, Fig 5k
set_size(6,3)
p1 <- Heatmap(Heatmap_chemokine[top_diff,], 
        column_split = factor(chemokine_metadata$Organ),
        cluster_columns = TRUE,
        show_column_dend = FALSE,
        cluster_column_slices = TRUE,
        cluster_rows = FALSE,
        show_row_dend = FALSE,
        col = col_fun,
        show_column_names = TRUE,
        show_row_names = TRUE, rect_gp = gpar(col = "white", lwd = 2))
p1

# Supp fig 8 f, g, h
set_size(3,2.5)

dodge_width <- 0.5
dodge <- position_dodge(width = dodge_width)
chemokine_plot <- chemokine_metadata %>% filter(sample_id != "HT413C1" & sample_id != "CM663C1" ) %>% group_by(Organ,  sample_id, site) %>% summarise(level = median(CCL16)) #or CXCL12 or CCL18
p1 <- chemokine_plot %>%  ggplot(aes(x=site, y=level, fill = Organ, color = Organ, group = interaction(site, Organ))) +   geom_boxplot(position = dodge, width = 0.5,  alpha = 0.4)   + scale_fill_manual(values=c('brown', 'steelblue1')) +  scale_color_manual(values=c('brown', 'steelblue1')) + geom_line(aes(group = interaction(sample_id, Organ), color = Organ), position = dodge)  +  theme(axis.text=element_text(size=15)) + theme_classic() 


# Distance from CXCL12 expressing cells was available a value for each barcode in the sample (in 10 microns) from CXCL12 expressing cell (expressing atleast 3 transcripts of CXCL12)
dist <- read.csv("path/to/distance.csv")
dist <- dist %>% filter(group >=  0.5 & group <= 9) #group is the colname for distance
dist$bin <- cut(dist$group, breaks = 8)
unique(merged$bin)               